In [1]:
import os
import utils.args_parser  as argtools
import pytorch_lightning as pl
import numpy as np


# LOAD CONFIG

In [2]:
use_custom_dataset = True

### Option 1: Datasets from the paper

In [3]:
if not use_custom_dataset:
    print('Using dataset from the paper')
    dataset_file =  os.path.join('_params', 'dataset_adult.yaml')
    model_file =   os.path.join('_params', 'model_carefl.yaml')
    trainer_file =   os.path.join('_params', 'trainer.yaml')

    yaml_file = ''
    
    if yaml_file == '':
        cfg = argtools.parse_args(dataset_file)
        cfg.update(argtools.parse_args(model_file))
        cfg.update(argtools.parse_args(trainer_file))
    else:
        cfg = argtools.parse_args(yaml_file)

### Option 2: New dataset

In [4]:
if use_custom_dataset:
    print('Using custom dataset')
    model_file =   os.path.join('_params', 'model_carefl.yaml')
    trainer_file =   os.path.join('_params', 'trainer.yaml')

    yaml_file = ''
    if yaml_file == '':
        cfg = argtools.parse_args(model_file)
        cfg.update(argtools.parse_args(trainer_file))
    else:
        cfg = argtools.parse_args(yaml_file)



Using custom dataset


In [5]:

# Config for new dataset

cfg['dataset'] = {
    'name': '2nodes',
    'params1': {},
    'params2': {}
}

cfg['dataset']['params1'] = {
    'data_dir': '../Data',
    'batch_size': 1000,
    'num_workers': 0
}

cfg['dataset']['params2'] = {
    'num_samples_tr': 5000,
    'equations_type': 'linear',
    'normalize': 'lik',
    'likelihood_names': 'd',
    'lambda_': 0.05,
    'normalize_A': None,
}

### You can also update any parameter manually

In [6]:
    
cfg['root_dir'] = 'results'
cfg['seed'] = 10
pl.seed_everything(cfg['seed'])

cfg['dataset']['params'] = cfg['dataset']['params1'].copy()
cfg['dataset']['params'].update(cfg['dataset']['params2'])

cfg['dataset']['params']['data_dir'] = ''

cfg['trainer']['limit_train_batches'] = 1.0
cfg['trainer']['limit_val_batches'] = 1.0
cfg['trainer']['limit_test_batches'] = 1.0
cfg['trainer']['check_val_every_n_epoch'] = 10


def print_if_not_dict(key, value, extra=''):
    if not isinstance(value, dict):
        print(f"{extra}{key}: {value}")
        return True
    else:
        print(f"{extra}{key}:")
        False
        
for key, value in cfg.items():
    if not print_if_not_dict(key, value):
        for key2, value2 in value.items():
            if not print_if_not_dict(key2, value2, extra='\t'):
                for key3, value3 in value2.items():
                    print_if_not_dict(key3, value3, extra='\t\t')


Global seed set to 10


optimizer:
	name: adam
	params:
		lr: 0.005
		betas: [0.9, 0.999]
		weight_decay: 1.2e-06
scheduler:
	name: exp_lr
	params:
		gamma: 0.99
model:
	name: carefl
	params:
		flow_architecture: spline
		flow_net_class: mlp
		distr_z: normal
		n_layers: 4
		n_hidden: 10
seed: 10
root_dir: results
early_stopping: True
trainer:
	max_epochs: 200
	min_epochs: 50
	limit_train_batches: 1.0
	limit_val_batches: 1.0
	limit_test_batches: 1.0
	check_val_every_n_epoch: 10
	progress_bar_refresh_rate: 1
	flush_logs_every_n_steps: 100
	log_every_n_steps: 2
	precision: 32
	terminate_on_nan: True
	auto_select_gpus: True
	deterministic: True
	weights_summary: None
	gpus: None
	num_sanity_val_steps: 2
	track_grad_norm: -1
	gradient_clip_val: 0.0
dataset:
	name: 2nodes
	params1:
		data_dir: ../Data
		batch_size: 1000
		num_workers: 0
	params2:
		num_samples_tr: 5000
		equations_type: linear
		normalize: lik
		likelihood_names: d
		lambda_: 0.05
		normalize_A: None
	params:
		data_dir: 
		batch_size: 1000
		num_

# LOAD DATASET

In [7]:
from utils.constants import Cte


print('These are datasets from the paper:')
for data_name in Cte.DATASET_LIST:
    print(f"\t{data_name}")
    


print(f"\nUsing dataset: {cfg['dataset']['name']}")

These are datasets from the paper:
	collider
	triangle
	loan
	mgraph
	chain
	adult
	german

Using dataset: 2nodes


In [8]:
# import os

# import pytorch_lightning as pl
# import torch
# from sklearn import preprocessing
# import torch_geometric
# #from torch_geometric.data import DataLoader
# # from torch_geometric.utils import degree
# # from torchvision import transforms as transform_lib

# # from data_modules._scalers import MaskedTensorLikelihoodScaler
# # from data_modules._scalers import MaskedTensorStandardScaler
# # from datasets.transforms import ToTensor
# # from utils.constants import Cte



# # from datasets.toy import create_toy_dataset
# # from utils.distributions import *

In [9]:
if cfg['dataset']['name'] in Cte.DATASET_LIST:
    from data_modules.het_scm import HeterogeneousSCMDataModule

    dataset_params = cfg['dataset']['params'].copy()
    dataset_params['dataset_name'] = cfg['dataset']['name']

    data_module = HeterogeneousSCMDataModule(**dataset_params)
    data_module.prepare_data()

elif cfg['dataset']['name']  == '2nodes':
    from data_modules.my_toy_scm import MyToySCMDataModule
    from utils.distributions import *
    
    dataset_params = cfg['dataset']['params'].copy()
    dataset_params['dataset_name'] = cfg['dataset']['name']
    
    dataset_params['nodes_to_intervene'] = ['x1']
    dataset_params['structural_eq'] = {'x1': lambda u1: u1,
                                            'x2': lambda u2, x1: u2 + x1}
    dataset_params['noises_distr'] = {'x1': Normal(0,1),
                                           'x2': Normal(0,1)}
    dataset_params['adj_edges'] = {'x1': ['x2'],
                                        'x2': []}
    
    data_module = MyToySCMDataModule(**dataset_params)
    data_module.prepare_data()
    

# LOAD MODEL

In [10]:
data_loader = data_module.train_dataloader()

#data_module.batch_size = bs

batch = next(iter(data_loader))

/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/workplace/prchao/VACA/datasets/transforms.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample)


In [11]:
batch

DataBatch(x=[2000, 1], edge_index=[2, 3000], edge_attr=[3000, 3], u=[1000, 2], mask=[2000, 1], node_ids=[2000, 2], num_nodes=2000, batch=[2000], ptr=[1001])

In [12]:
model = None
model_params = cfg['model']['params'].copy()

print(f"\nUsing model: {cfg['model']['name']}")


Using model: carefl


In [13]:

# VACA
if cfg['model']['name'] == Cte.VACA:
    from models.vaca.vaca import VACA

    model_params['is_heterogeneous'] = data_module.is_heterogeneous
    model_params['likelihood_x'] = data_module.likelihood_list

    model_params['deg'] = data_module.get_deg(indegree=True)
    model_params['num_nodes'] = data_module.num_nodes
    model_params['edge_dim'] = data_module.edge_dimension
    model_params['scaler'] = data_module.scaler

    model = VACA(**model_params)
    model.set_random_train_sampler(data_module.get_random_train_sampler())
# VACA with PIWAE
elif cfg['model']['name'] == Cte.VACA_PIWAE:
    from models.vaca.vaca_piwae import VACA_PIWAE

    model_params['is_heterogeneous'] = data_module.is_heterogeneous

    model_params['likelihood_x'] = data_module.likelihood_list

    model_params['deg'] = data_module.get_deg(indegree=True)
    model_params['num_nodes'] = data_module.num_nodes
    model_params['edge_dim'] = data_module.edge_dimension
    model_params['scaler'] = data_module.scaler

    model = VACA_PIWAE(**model_params)
    model.set_random_train_sampler(data_module.get_random_train_sampler())



# MultiCVAE
elif cfg['model']['name'] == Cte.MCVAE:
    from models.multicvae.multicvae import MCVAE

    model_params['likelihood_x'] = data_module.likelihood_list

    model_params['topological_node_dims'] = data_module.train_dataset.get_node_columns_in_X()
    model_params['topological_parents'] = data_module.topological_parents
    model_params['scaler'] = data_module.scaler
    model_params['num_epochs_per_nodes'] = int(
        np.floor((cfg['trainer']['max_epochs'] / len(data_module.topological_nodes))))
    model = MCVAE(**model_params)
    model.set_random_train_sampler(data_module.get_random_train_sampler())
    cfg['early_stopping'] = False

# CAREFL
elif cfg['model']['name'] == Cte.CARELF:
    from models.carefl.carefl import CAREFL

    model_params['node_per_dimension_list'] = data_module.train_dataset.node_per_dimension_list
    model_params['scaler'] = data_module.scaler
    model = CAREFL(**model_params)

In [14]:

model.summarize()
model.set_optim_params(optim_params=cfg['optimizer'],
                       sched_params=cfg['scheduler'])

/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/pytorch_lightning/core/lightning.py:1726: LightningDeprecationWarning: Argument `mode` in `LightningModule.summarize` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=1` to replicate `mode=top` behavior.
  rank_zero_deprecation(

  | Name       | Type                 | Params
----------------------------------------------------
0 | flow_model | NormalizingFlowModel | 752   
----------------------------------------------------
752       Trainable params
0         Non-trainable params
752       Total params
0.003     Total estimated model params size (MB)


# LOAD EVALUATOR

In [15]:
from models._evaluator import MyEvaluator

evaluator = MyEvaluator(model=model,
                        intervention_list=data_module.train_dataset.get_intervention_list(),
                        scaler=data_module.scaler
                        )
model.set_my_evaluator(evaluator=evaluator)



In [16]:
data_module.train_dataset.get_intervention_list()

[({'x1': -1.02}, '-1_sigma'),
 ({'x1': -0.52}, '-0.5_sigma'),
 ({'x1': -0.02}, '0_sigma'),
 ({'x1': 0.47}, '0.5_sigma'),
 ({'x1': 0.97}, '1_sigma')]

In [17]:
for intervention in data_module.train_dataset.get_intervention_list():
    inter_dict, name = intervention
    print(f'Intervention name: {name}')
    for node_name, value in inter_dict.items():
        print(f"\t{node_name}: {value}")

Intervention name: -1_sigma
	x1: -1.02
Intervention name: -0.5_sigma
	x1: -0.52
Intervention name: 0_sigma
	x1: -0.02
Intervention name: 0.5_sigma
	x1: 0.47
Intervention name: 1_sigma
	x1: 0.97


# PREPARE TRAINING

In [18]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger


is_training = True
load = True

print(f'Is training activated? {is_training}')
print(f'Is loading activated? {load}')


Is training activated? True
Is loading activated? True


In [19]:
if yaml_file == '':
    if (cfg['dataset']['name'] in [Cte.GERMAN]) and (cfg['dataset']['params3']['train_kfold'] == True):
        save_dir = argtools.mkdir(os.path.join(cfg['root_dir'],
                                               argtools.get_experiment_folder(cfg),
                                               str(cfg['seed']), str(cfg['dataset']['params3']['kfold_idx'])))
    else:
        save_dir = argtools.mkdir(os.path.join(cfg['root_dir'],
                                               argtools.get_experiment_folder(cfg),
                                               str(cfg['seed'])))
else:
    save_dir = os.path.join(*yaml_file.split('/')[:-1])
print(f'Save dir: {save_dir}')


Save dir: results/2nodes_5000_linear_lik_d_0.05_None/carefl/spline_mlp_normal_4_10/adam/0.005_0.9_0.999_1.2e-06_exp_lr_0.99/10


In [20]:
logger = TensorBoardLogger(save_dir=save_dir, name='logs', default_hp_metric=False)

out = logger.log_hyperparams(argtools.flatten_cfg(cfg))

save_dir_ckpt = argtools.mkdir(os.path.join(save_dir, 'ckpt'))
if load:
    ckpt_file = argtools.newest(save_dir_ckpt)
else:
    ckpt_file = None
callbacks = []

print(f"ckpt_file: {ckpt_file}")

ckpt_file: None


In [21]:
if is_training:
    checkpoint = ModelCheckpoint(period=1,
                                 monitor=model.monitor(),
                                 mode=model.monitor_mode(),
                                 save_top_k=1,
                                 save_last=True,
                                 filename='checkpoint-{epoch:02d}',
                                 dirpath=save_dir_ckpt)
    callbacks = [checkpoint]

    
    if cfg['early_stopping']:
        early_stopping = EarlyStopping(model.monitor(), mode=model.monitor_mode(), min_delta=0.0, patience=50)
        callbacks.append(early_stopping)
    trainer = pl.Trainer(logger=logger, callbacks=callbacks, **cfg['trainer'])
    
if load:
    if ckpt_file is None:
        print(f'No ckpt files in {save_dir_ckpt}')
    else:
        print(f'\nLoading from: {ckpt_file}')
        if is_training:
            trainer = pl.Trainer(logger=logger, callbacks=callbacks, resume_from_checkpoint=ckpt_file,
                             **cfg['trainer'])
        else:

            model = model.load_from_checkpoint(ckpt_file, **model_params)
            evaluator.set_model(model)
            model.set_my_evaluator(evaluator=evaluator)

            if cfg['model']['name'] in [Cte.VACA_PIWAE, Cte.VACA, Cte.MCVAE]:
                model.set_random_train_sampler(data_module.get_random_train_sampler())


/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:487: LightningDeprecationWarning: Argument `period` in `ModelCheckpoint` is deprecated in v1.3 and will be removed in v1.5. Please use `every_n_epochs` instead.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No ckpt files in results/2nodes_5000_linear_lik_d_0.05_None/carefl/spline_mlp_normal_4_10/adam/0.005_0.9_0.999_1.2e-06_exp_lr_0.99/10/ckpt


/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


In [22]:
path = os.path.join(save_dir,"logs")

if not os.path.exists(path):
    os.makedirs(path)

In [23]:
if is_training:
    trainer.fit(model, data_module)
    # save_yaml(model.get_arguments(), file_path=os.path.join(save_dir, 'hparams_model.yaml'))
    argtools.save_yaml(cfg, file_path=os.path.join(save_dir, 'hparams_full.yaml'))
    # %% Testing

Validation sanity check: 0it [00:00, ?it/s]

/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/workplace/prchao/VACA/datasets/transforms.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample)
/workplace/prchao/VACA/data_modules/_scalers.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clo

Training: -1it [00:00, ?it/s]

/home/prchao/miniconda3/envs/vaca/lib/python3.9/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:283: LightningDeprecationWarning: The signature of `ModelHooks.on_train_epoch_end` has changed in v1.3. `outputs` parameter has been deprecated. Support for the old signature will be removed in v1.5
  self._warning_cache.deprecation(


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving yaml: results/2nodes_5000_linear_lik_d_0.05_None/carefl/spline_mlp_normal_4_10/adam/0.005_0.9_0.999_1.2e-06_exp_lr_0.99/10/hparams_full.yaml


# TESTING

In [24]:
# model_parameters = filter(lambda p: p.requires_grad, model.parameters())
# params = int(sum([p.numel() for p in model_parameters]))

# model.eval()
# model.freeze()  # IMPORTANT

In [25]:
# output_valid = model.evaluate(dataloader=data_module.val_dataloader(),
#                               name='valid',
#                               save_dir=save_dir,
#                               plots=False)

In [26]:
# path = os.path.join(save_dir,"images")

# if not os.path.exists(path):
#     os.makedirs(path)

In [27]:
# output_test = model.evaluate(dataloader=data_module.test_dataloader(),
#                              name='test',
#                              save_dir=save_dir,
#                              plots=True)
# output_valid.update(output_test)

# output_valid.update(argtools.flatten_cfg(cfg))


In [28]:
# import json
# output_valid.update({'ckpt_file': ckpt_file,
#                      'num_parameters': params})

# with open(os.path.join(save_dir, 'output.json'), 'w') as f:
#     json.dump(output_valid, f)
# print(f'Experiment folder: {save_dir}')

# Custom interventions

In [29]:
bs = data_module.batch_size
data_module.batch_size = 3
x_I = {'x1': 10}  # Intervention before normalizing
data_loader = data_module.test_dataloader()
data_loader.dataset.set_intervention(x_I)
data_loader = data_module.test_dataloader()
data_module.batch_size = bs

batch = next(iter(data_loader))



print(batch)


DataBatch(x=[6, 1], edge_index=[2, 9], edge_attr=[9, 3], u=[3, 2], mask=[6, 1], node_ids=[6, 2], x_i=[6, 1], edge_index_i=[2, 9], edge_attr_i=[9, 3], num_nodes=6, batch=[6], ptr=[4])


In [30]:
batch.x_i

tensor([[10.1029],
        [ 2.2218],
        [10.1029],
        [ 0.0525],
        [10.1029],
        [ 1.2740]])

In [31]:
x_hat, z = model.get_intervention(batch,
                         x_I=data_loader.dataset.x_I,
                         nodes_list=data_loader.dataset.nodes_list,
                         return_type = 'sample', # mean or sample
                         use_aggregated_posterior = False,
                         normalize = True)

print(f"Original: {batch.x.flatten()}")
print(f"Intervened: {batch.x_i.flatten()}")
print(f"Reconstructed: {x_hat.flatten()}")



AttributeError: 'CAREFL' object has no attribute 'get_intervention'

# Custom counterfactuals

In [ ]:
data_loader

In [ ]:
bs = data_module.batch_size
data_module.batch_size = 1
x_I = {'x1': 10.0}  # Intervention before normalizing
data_loader = data_module.test_dataloader()
data_loader.dataset.set_intervention(x_I,is_noise=False)
data_loader = data_module.test_dataloader()
data_module.batch_size = bs

batch = next(iter(data_loader))



# print(batch)
# x1-> x2 
# x2 = x1 + N(0,1)

In [ ]:
batch

In [ ]:
x_I = {'x1': 10.0}
vaca_pred, gt_cf, factual = model.get_counterfactual_distr(data_loader,
                                        x_I=x_I,
                                        is_noise = False,
                                        num_batches= 1,
                                        normalize=False,
                                        )

vaca_pred['all'], factual

In [ ]:
gt_cf